# 🔬 ISIC Cilt Lezyonu Analizi - Hatasız Versiyon

**Mustafa Engin Dalgıç | 254309502 | Üsküdar Üniversitesi**

---

## ▶️ ÇALIŞTIRMA:

1. **Sağ üstten "Bağlan" (Connect) tıklayın**
2. **Runtime > Run all** veya her hücreyi **Shift+Enter** ile çalıştırın

---

In [ ]:
# ==================== BAŞLANGIÇ ====================
print("🚀 Program başlıyor...")
print("Lütfen bekleyin...\n")

In [ ]:
# ==================== KÜTÜPHANELER ====================
print("📦 Kütüphaneler yükleniyor...\n")

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# OpenCV
try:
    import cv2
    print(f"✅ OpenCV {cv2.__version__}")
except:
    print("⚠️ OpenCV kuruluyor...")
    !pip install opencv-python -q
    import cv2
    print(f"✅ OpenCV {cv2.__version__} kuruldu")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}")
print(f"✅ Matplotlib hazır")
print("\n✅ Tüm kütüphaneler hazır!\n")

In [ ]:
# ==================== PROJE İNDİRME ====================
print("📥 GitHub'dan proje indiriliyor...\n")

!rm -rf UU_DIP_ISIC_SKIN_LESION_ANALYSIS 2>/dev/null
!git clone https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS.git -q

%cd UU_DIP_ISIC_SKIN_LESION_ANALYSIS

print("✅ Proje indirildi!\n")

In [ ]:
# ==================== VERİ SETİ KONTROLÜ ====================
print("🔍 Veri seti kontrol ediliyor...\n")

DATA_PATH = "ISIC"

# Tüm görüntüleri bul
image_files = []
if os.path.exists(DATA_PATH):
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_files.append(os.path.join(root, file))

if len(image_files) == 0:
    print("❌ HATA: ISIC klasöründe görüntü bulunamadı!")
    print("\n💡 Çözüm: GitHub reposuna ISIC klasörünü ve içine görüntüleri ekleyin.")
    raise Exception("Veri seti bulunamadı")
else:
    print(f"✅ {len(image_files)} görüntü bulundu!")
    print(f"\n📁 İlk 5 dosya:")
    for i, f in enumerate(image_files[:5], 1):
        print(f"   {i}. {os.path.basename(f)}")
    if len(image_files) > 5:
        print(f"   ... ve {len(image_files)-5} tane daha\n")

In [ ]:
# ==================== VERİ YÜKLEME (HATA KONTROLÜ İLE) ====================
print("📊 Veri seti yükleniyor...\n")

def load_images_safe(image_paths, max_samples=None):
    """Güvenli veri yükleme - hata kontrolü ile"""
    data = []
    
    if max_samples:
        image_paths = image_paths[:max_samples]
    
    for i, img_path in enumerate(image_paths):
        try:
            img = Image.open(img_path)
            width, height = img.size
            
            # Kanal sayısı
            if img.mode == 'RGB':
                channels = 3
            elif img.mode == 'L':
                channels = 1
            elif img.mode == 'RGBA':
                channels = 4
            else:
                channels = len(img.getbands())
            
            file_size = os.path.getsize(img_path) / 1024  # KB
            
            data.append({
                'filename': os.path.basename(img_path),
                'filepath': img_path,
                'width': width,
                'height': height,
                'channels': channels,
                'file_size_kb': round(file_size, 2)
            })
            
            if (i+1) % 50 == 0:
                print(f"   {i+1}/{len(image_paths)} görüntü yüklendi...")
                
        except Exception as e:
            print(f"   ⚠️ Atlandı: {os.path.basename(img_path)} - {e}")
            continue
    
    return pd.DataFrame(data)

# Veriyi yükle (hızlı test için ilk 100 görüntü)
train_df = load_images_safe(image_files, max_samples=100)

if len(train_df) == 0:
    print("\n❌ HATA: Hiçbir görüntü yüklenemedi!")
    raise Exception("Veri yükleme başarısız")
else:
    print(f"\n✅ {len(train_df)} görüntü başarıyla yüklendi!")
    print("\n📋 İlk 5 satır:")
    display(train_df.head())

In [ ]:
# ==================== İSTATİSTİKLER ====================
print("\n📊 Veri seti istatistikleri:\n")

print(f"📐 Çözünürlük:")
print(f"   Ortalama: {train_df['width'].mean():.0f}x{train_df['height'].mean():.0f} px")
print(f"   Min: {train_df['width'].min()}x{train_df['height'].min()}")
print(f"   Max: {train_df['width'].max()}x{train_df['height'].max()}")

print(f"\n💾 Dosya Boyutu:")
print(f"   Ortalama: {train_df['file_size_kb'].mean():.2f} KB")
print(f"   Toplam: {train_df['file_size_kb'].sum()/1024:.2f} MB")

print(f"\n🎨 Kanal Dağılımı:")
channel_counts = train_df['channels'].value_counts()
for ch, count in channel_counts.items():
    ch_name = {1: 'Grayscale', 3: 'RGB', 4: 'RGBA'}.get(ch, f'{ch} Kanal')
    print(f"   {ch_name}: {count} görüntü")

In [ ]:
# ==================== İLK GÖRÜNTÜ ====================
print("\n🖼️ İlk görüntü gösteriliyor...\n")

def resize_image(img, max_size=500):
    """Görüntüyü küçült"""
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        new_w = int(w * scale)
        new_h = int(h * scale)
        return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return img

# İlk görüntüyü yükle
first_img = train_df.iloc[0]
img = cv2.imread(first_img['filepath'])
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_rgb = resize_image(img_rgb)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

# Göster
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('✅ İLK GÖRÜNTÜ', fontsize=14, fontweight='bold')

axes[0].imshow(img_rgb)
axes[0].set_title(f"RGB - {first_img['filename'][:30]}")
axes[0].axis('off')

axes[1].imshow(img_gray, cmap='gray')
axes[1].set_title('Grayscale')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"✅ Görüntü gösterildi: {first_img['filename']}")
print(f"📐 Boyut: {img_rgb.shape[1]}x{img_rgb.shape[0]} px")

In [ ]:
# ==================== RASTGELE 4 GÖRÜNTÜ ====================
print("\n📸 Rastgele görüntüler gösteriliyor...\n")

# Rastgele 4 görüntü seç
np.random.seed(42)
n_samples = min(4, len(train_df))
random_idx = np.random.choice(train_df.index, size=n_samples, replace=False)
samples = train_df.iloc[random_idx]

# Göster
fig, axes = plt.subplots(n_samples, 2, figsize=(10, n_samples * 2.5))
fig.suptitle('RGB vs Grayscale Karşılaştırma', fontsize=14, fontweight='bold')

for idx, (i, row) in enumerate(samples.iterrows()):
    img = cv2.imread(row['filepath'])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = resize_image(img_rgb)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    axes[idx, 0].imshow(img_rgb)
    axes[idx, 0].set_title(f"RGB - {row['filename'][:25]}")
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(img_gray, cmap='gray')
    axes[idx, 1].set_title(f"Grayscale - {row['width']}x{row['height']}")
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Görüntüler gösterildi!")

In [ ]:
# ==================== HİSTOGRAM ====================
print("\n📊 Histogram analizi...\n")

# İlk görüntü için histogram
hist_r = cv2.calcHist([img_rgb], [0], None, [256], [0, 256])
hist_g = cv2.calcHist([img_rgb], [1], None, [256], [0, 256])
hist_b = cv2.calcHist([img_rgb], [2], None, [256], [0, 256])
hist_gray = cv2.calcHist([img_gray], [0], None, [256], [0, 256])

# Göster
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
fig.suptitle('Histogram Analizi', fontsize=14, fontweight='bold')

# RGB
axes[0].plot(hist_r, color='red', alpha=0.7, label='Red')
axes[0].plot(hist_g, color='green', alpha=0.7, label='Green')
axes[0].plot(hist_b, color='blue', alpha=0.7, label='Blue')
axes[0].set_title('RGB Histogram')
axes[0].set_xlabel('Piksel Değeri')
axes[0].set_ylabel('Frekans')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Grayscale
axes[1].plot(hist_gray, color='black')
axes[1].set_title('Grayscale Histogram')
axes[1].set_xlabel('Piksel Değeri')
axes[1].set_ylabel('Frekans')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Histogram gösterildi!")

In [ ]:
# ==================== CLAHE ====================
print("\n✨ CLAHE (kontrast iyileştirme)...\n")

# CLAHE uygula
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img_clahe = clahe.apply(img_gray)

# Göster
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('CLAHE - Kontrast İyileştirme', fontsize=14, fontweight='bold')

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Orijinal')
axes[0].axis('off')

axes[1].imshow(img_clahe, cmap='gray')
axes[1].set_title('CLAHE Uygulanmış')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✅ CLAHE uygulandı!")

---

## 🎉 TEST TAMAMLANDI!

### ✅ Başarıyla Tamamlanan İşlemler:

1. ✅ Veri seti yüklendi
2. ✅ İstatistikler çıkarıldı
3. ✅ Görüntüler gösterildi
4. ✅ Histogram analizi yapıldı
5. ✅ CLAHE uygulandı

### 🚀 TAM PROGRAMI ÇALIŞTIRMAK İÇİN:

Aşağıdaki hücreyi çalıştırın (10-15 dakika sürer):

In [ ]:
# TAM PROGRAM (45+ grafik)
print("🚀 TAM PROGRAM BAŞLIYOR...")
print("⏱️ 10-15 dakika sürecek.\n")

!python isic_project.py

---

### 💬 İletişim

**Mustafa Engin Dalgıç**  
📧 engindalgic86@gmail.com  
🎓 Üsküdar Üniversitesi  
🔗 [GitHub Repo](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)

---

⭐ **Projeyi beğendiyseniz GitHub'da yıldız vermeyi unutmayın!** ⭐